# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [11]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [12]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-4o", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2024-05-23 08:30:51,049 - 8021201664 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

2024-05-23 08:33:30,416 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')


 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...


2024-05-23 08:33:35,536 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 08:33:35,537 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 08:33:35,537 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 08:33:35,538 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_

To continuously monitor and analyze market data for AAPL, we need to leverage the expertise of our team members. We will break down the task into smaller, manageable parts and delegate them accordingly.

1. **Data Collection and Initial Analysis**: The Data Analyst can gather and preprocess the market data for AAPL.
2. **Statistical Modeling**: The Trading Strategy Developer can use statistical models to identify trends.
3. **Machine Learning**: The Data Analyst can also work on applying machine learning algorithms to predict market movements.
4. **Risk Assessment**: The Risk Advisor can evaluate the identified trends and predictions for potential risks.
5. **Insights and Alerts**: The Trade Advisor can provide insights and alerts based on the analysis.

Let's start by delegating the tasks.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Collect and preprocess market data for AAPL. This includes gathering historical price data, trading volumes, a

2024-05-23 08:33:39,602 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 08:33:39,602 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 08:33:39,603 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 08:33:39,603 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_

...

 


Search results: Title: Apple Inc. (AAPL) Stock Historical Prices & Data - Yahoo Finance
Link: https://finance.yahoo.com/quote/AAPL/history/
Snippet: Discover historical prices for AAPL stock on Yahoo Finance. View daily, weekly or monthly format back to when Apple Inc. stock was issued.
---
Title: Apple (AAPL) Historical Data - Nasdaq
Link: https://www.nasdaq.com/market-activity/stocks/aapl/historical
Snippet: Find the latest historical data for Apple Inc. Common Stock (AAPL) at Nasdaq.com. View historical data in a monthly, bi-annual, or yearly format.
---
Title: AAPL | Apple Inc. Stock Price & News - WSJ
Link: https://www.wsj.com/market-data/quotes/AAPL/historical-prices
Snippet: Apple Inc. historical stock charts and prices, analyst ratings, financials, and today's real-time AAPL stock price.
---
Title: Stock Price - Apple Investor Relations
Link: https://investor.apple.com/stock-price/default.aspx
Snippet: Stock Quote: NASDAQ: AAPL ; Day's Open192.26 ; Closing Price190.90 ; Vol

2024-05-23 08:43:05,172 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 08:43:05,173 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 08:43:05,173 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 08:43:05,173 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_

Thought: 
I have found multiple sources providing historical data for AAPL, including Yahoo Finance, Nasdaq, WSJ, Barchart, MarketWatch, Investing.com, and more. I will use one of these sources to gather detailed historical data, including price movements, trading volumes, and technical indicators relevant to the trading strategies mentioned.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL/history/"} 

Apple Inc. (AAPL) Stock Historical Prices & Data - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How

2024-05-23 08:43:29,635 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')


Thought: 
I have accessed the historical price movements and trading volumes for AAPL from Yahoo Finance. Now I will compile the historical data and insights relevant to the breakout trading, mean reversion, momentum trading, and scalping strategies for AAPL, including price movements, trading volumes, and any pertinent technical indicators.

I now know the final answer.

Final Answer: 

### Detailed Execution Plans for AAPL

#### Strategy 1: Breakout Trading
- **Entry Point:** Place a buy order slightly above the resistance level of $181.93 to capitalize on a breakout scenario. Given current market conditions, also consider entering trades closer to the lower end of the day's range (~$190.27) as it aligns with recent support levels.
- **Stop Loss:** Set a stop loss just below the support level at $178.00 to minimize potential losses. Recent data suggests placing it closer to $188.00 due to increased volatility.
- **Take Profit:** Consider taking profit incrementally at intervals (e.g.

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
Markdown(result)

### Comprehensive Risk Analysis Report for AAPL Trading Strategies

#### Strategy 1: Breakout Trading

- **Entry Point:** Buy slightly above the resistance level of $181.93 to capture a breakout. Current conditions suggest considering trades closer to the lower end of the day's range (~$190.27) aligning with recent support levels.
- **Stop Loss:** Set just below the support level at $178.00 to minimize losses. Recent data suggests placing it closer to $188.00 due to increased volatility.
- **Take Profit:** Incremental profit taking at $1 intervals or target around $185.00. Adjust to $192.00 based on the day's range ($190.27 - $192.82) to lock in gains before potential reversals.
- **Volume Confirmation:** Ensure significant trading volume to validate breakouts.

**Risk Analysis and Mitigation:**
- **Risk:** False breakouts leading to losses, especially in low volume scenarios.
- **Mitigation:** Implement strict volume confirmation rules, consider partial entries to reduce exposure, and use trailing stops to lock in gains.

#### Strategy 2: Mean Reversion

- **Entry Point:** Buy if the stock dips towards support ($178.14), expecting a reversion towards the mean or resistance. Current conditions suggest entering at ~$190.27.
- **Stop Loss:** Slightly below the support level, e.g., $177.50. Adjust to $188.00 based on recent support.
- **Take Profit:** Target midpoint of the range (~$180.00) or resistance at $181.93. Consider $192.00 as the upper target given the day's range.
- **Backtesting and Diversification:** Use historical data for backtesting and diversify across assets to spread risk.

**Risk Analysis and Mitigation:**
- **Risk:** Prolonged deviations from the mean, leading to extended drawdowns.
- **Mitigation:** Use robust backtesting to validate strategy reliability, diversify trades, and employ dynamic stop-loss adjustments.

#### Strategy 3: Momentum Trading

- **Entry Point:** Enter trades on significant upward momentum, confirmed by high trading volume and bullish technical indicators (e.g., MACD crossover, RSI above 70). Current conditions suggest entering at ~$190.90.
- **Stop Loss:** Dynamic stop loss at 2% below entry (~$187.08) to protect against reversals.
- **Take Profit:** Use trailing stops to lock in profits as the stock price increases. Aim for $192.82 and adjust based on momentum.
- **Regular Monitoring:** Maintain discipline and monitor market movements to avoid missing key entry and exit points.

**Risk Analysis and Mitigation:**
- **Risk:** Sudden momentum shifts resulting in rapid losses.
- **Mitigation:** Implement dynamic stop-loss mechanisms, use momentum indicators to confirm entries, and maintain constant market monitoring.

#### Strategy 4: Scalping

- **Entry Point:** Enter trades based on short-term technical signals within the day's range ($190.27 - $192.82). Current conditions suggest entering near $190.27.
- **Stop Loss:** Tight stop loss at 0.5% below entry (~$189.32) to limit losses.
- **Take Profit:** Small profit targets (0.5%-1%) with multiple trades throughout the day. Aim for $191.37 to $192.00 based on current conditions.
- **Efficient Execution:** Use a low-latency trading platform with high-speed execution to minimize execution risk.

**Risk Analysis and Mitigation:**
- **Risk:** High-frequency trades leading to increased transaction costs and potential slippage.
- **Mitigation:** Use high-speed trading platforms, monitor transaction costs, and implement strict entry/exit criteria to enhance trade efficiency.

### General Risk Management & Considerations
- **Position Sizing:** Align position sizes with risk tolerance, typically risking no more than 1-2% of trading capital per trade.
- **Market Conditions:** Stay updated with news or events impacting AAPL’s stock price, especially around earnings dates (Aug 1, 2024 - Aug 5, 2024).
- **Economic Moat:** Consider Apple’s long-term economic moat for stability and resilience against market volatility.

By integrating these strategies and adjusting them based on real-time market conditions and risk tolerance, traders can potentially capitalize on AAPL’s price movements while managing risk effectively.